In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

from gymnax.environments.classic_control.cartpole import EnvParams, CartPole
from rejax import get_algo
from rejax.evaluate import evaluate, EvalState
from tqdm.notebook import tqdm

import _pickle as pickle
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import yaml


In [ ]:
config_path = "./configs/custom/cartpole.yaml"

with open(config_path, "r") as f:
    config = yaml.safe_load(f.read())

In [ ]:
algo_name = "ppo"
config = config[algo_name]
num_seeds = 500

num_files = 3
starting_seed_id = 1000


env = CartPole()
config["env"] = env
max_steps_in_episode = 200

In [ ]:
config

In [ ]:
for file_i in tqdm(range(num_files)):
    seed_id = starting_seed_id + file_i

    key = jax.random.PRNGKey(seed_id)
    keys = jax.random.split(key, num_seeds)

    gravities = jax.random.uniform(key, shape=(num_seeds,))

    algo_cls = get_algo(algo_name)
    algo = jax.vmap(
        lambda gravity: algo_cls.create(
            **config,
            env_params=EnvParams(
                gravity=gravity,
                max_steps_in_episode=max_steps_in_episode,
            ),
        )
    )(
        gravities
    )


    def eval_callback(algo, ts, rng):
        act = algo.make_act(ts)
        return evaluate(act, rng, env, algo.env_params, 50, max_steps_in_episode)

    def get_returns(algo, ts, rng):
        eval_info = eval_callback(algo, ts, rng)
        return eval_info.length, eval_info.return_

    algo = algo.replace(
        eval_callback=get_returns
    )

    # Train
    vmap_train = jax.jit(jax.vmap(algo_cls.train, in_axes=(0, 0)))
    ts, (_, returns) = vmap_train(algo, keys)
    returns.block_until_ready()

    # Get expert actions
    def get_expert_actions(ts, num_steps):
        act = algo.make_act(ts)
        def step(step_i: int, state: EvalState):
            rng, rng_act, rng_step = jax.random.split(state.rng, 3)
            action = act(state.trajectory["obss"][step_i], rng_act)

            state = EvalState(
                rng,
                None,
                None,
                trajectory=dict(
                    obss=state.trajectory["obss"],
                    actions=state.trajectory["actions"].at[step_i].set(action),
                ),
            )
            return state

        state = EvalState(
            key,
            None,
            None,
            trajectory=dict(
                obss=ts.store_buffer.data.obs,
                actions=ts.store_buffer.data.action,
            )
        )
        state = jax.lax.fori_loop(
            0,
            num_steps,
            step,
            state,
        )
        return state.trajectory["actions"]

    expert_actions = jax.vmap(
        jax.jit(get_expert_actions, static_argnames=["num_steps"]),
        in_axes=[0, None],
    )(
        ts,
        ts.store_buffer.data.action.shape[1],
    )

    # "Expert data" from last PPO iteration
    expert_trajectories = jax.vmap(eval_callback)(algo, ts, keys)

    # Save data
    pickle.dump(
        {
            "buffer_info": {k: v for k, v in ts.store_buffer.__dict__.items() if k != "data"},
            "learning_histories": {
                "expert_action": np.array(expert_actions),
                **{k: np.array(v) for k, v in ts.store_buffer.data._asdict().items()}
            },
            "algorithm": {
                "algo": algo_name,
                **{k: v for k, v in config.items() if k != "env"},
            },
            "env": type(config["env"]).__name__,
            "env_params": np.array(gravities),
            "observation_space": env.observation_space(EnvParams()),
            "action_space": env.action_space(EnvParams()),
            "expert_data": {
                k: np.array(v) for k, v in expert_trajectories.trajectory.items()
            },
        },
        open("{}/learning_hist-cartpole-num_tasks_{}-seed_{}-{}.pkl".format(
            "/home/bryanpu1/projects/aaai_2026/data/cartpole",
            num_seeds,
            seed_id,
            algo_name,
            ), "wb"),
    )


In [ ]:
assert 0

In [ ]:
regrets = (np.arange(ts.store_buffer.data.reward.shape[1])[None] + 1) * np.max(env_params, axis=-1, keepdims=True) - np.cumsum(ts.store_buffer.data.reward, axis=-1)

for regret in regrets[:5]:
    xrange = np.arange(len(regret))
    plt.plot(xrange, regret)
plt.show()


In [ ]:
for env_returns, env_param in zip(returns[:2], env_params):
    xrange = np.arange(len(env_returns)) * config["eval_freq"]
    regret = np.max(env_param, axis=-1) - env_returns
    print(regret.shape)
    mean = np.mean(regret, axis=-1)
    std = np.std(regret, axis=-1) / np.sqrt(regret.shape[-1])
    plt.plot(xrange, mean)
    plt.fill_between(xrange, mean - std, mean + std, alpha=0.2)
plt.show()


In [ ]:
for env_returns in returns[:10]:
    xrange = np.arange(len(env_returns)) * config["eval_freq"]
    mean = np.mean(env_returns, axis=-1)
    std = np.std(env_returns, axis=-1) / np.sqrt(env_returns.shape[-1])
    plt.plot(xrange, mean)
    plt.fill_between(xrange, mean - std, mean + std, alpha=0.2)
plt.show()
